# Computational Graph

# Requires_grad, Leaf

In [1]:
import torch
device = torch.device("cuda:0")
torch.cuda.set_device(device)

In [2]:
# User가 만든 ones, zeros, tensor 등은 requires_grad가 default로 False
# Gradient를 저장할 것이라는 뜻
a = torch.scalar_tensor(2, device=device)
b = torch.scalar_tensor(3, device=device)
c = torch.scalar_tensor(5, device=device, requires_grad=True)

In [3]:
out1 = a * b
out2 = out1 + c

In [4]:
out2.backward()

메모리 절약을 위해 grad는 최종 leaf에만 저장한다.

In [5]:
print(a.requires_grad)
print(b.requires_grad)
print(c.requires_grad)
print(out1.requires_grad)
print(out2.requires_grad)

False
False
True
False
True


In [6]:
print(a.is_leaf)
print(b.is_leaf)
print(c.is_leaf)
print(out1.is_leaf)
print(out2.is_leaf)

True
True
True
True
False


## requires_grad 속성의 전이

a, b가 requires_grad가 꺼져 있다면 가장 마지막으로 requires_grad가 True인 곳이 leaf가 된다.  
위의 경우 a, b에는 requires_grad 저장 X => out1도 leaf가 된다. 하지만 requires_grad는 False.

In [21]:
a = torch.scalar_tensor(2, device=device, requires_grad=True)
b = torch.scalar_tensor(3, device=device, requires_grad=True)
c = torch.scalar_tensor(5, device=device, requires_grad=True)
out1 = a * b
out2 = out1 + c

In [8]:
print(a.requires_grad)
print(b.requires_grad)
print(c.requires_grad)
print(out1.requires_grad)
print(out2.requires_grad)

True
True
True
True
True


In [9]:
print(a.is_leaf)
print(b.is_leaf)
print(c.is_leaf)
print(out1.is_leaf)
print(out2.is_leaf)

True
True
True
False
False


# Checking Grad

## Retain_grad

In [22]:
out2.backward()

In [23]:
print(a.grad)
print(b.grad)
print(c.grad)

# 기본적으로는 grad를 저장하지 않음 (leaf가 아니므로)
print(out1.grad)
print(out2.grad)

tensor(3., device='cuda:0')
tensor(2., device='cuda:0')
tensor(1., device='cuda:0')
None
None


In [17]:
out1.retain_grad()

In [19]:
print(a.grad)
print(b.grad)
print(c.grad)

print(out1.grad)
print(out2.grad)

tensor(3., device='cuda:0')
tensor(2., device='cuda:0')
tensor(2., device='cuda:0')
tensor(1., device='cuda:0')
None


## Hooking

Backward를 하면서 grad를 사용하고 원래는 버림(deallocation).  
Hooking = 버리기 전에 잠시 가져오는 것

In [24]:
def hooking(grad):
    print(grad)

In [28]:
a = torch.scalar_tensor(2, device=device, requires_grad=True)
b = torch.scalar_tensor(3, device=device, requires_grad=True)
c = torch.scalar_tensor(5, device=device, requires_grad=True)
out1 = a * b
out2 = out1 + c

In [29]:
out1.register_hook(hooking)

In [30]:
out2.backward()

tensor(1., device='cuda:0')


In [31]:
out1.grad 
# Hook하고 버렸기 때문에
# 여전히 None

# Backward는 어떻게 계산되는가?

## Linear Function ([Document](https://pytorch.org/docs/stable/notes/extending.html))


Grad를 계산하면서 graph의 선을 끊고 garbage collection을 진행한다. 메모리 이득.

In [37]:
a = torch.scalar_tensor(2, device=device, requires_grad=True)
b = torch.scalar_tensor(3, device=device, requires_grad=True)
c = torch.scalar_tensor(5, device=device, requires_grad=True)
out1 = a * b
out2 = out1 + c

In [36]:
out2.backward()

In [34]:
# 그래프가 부서져서 두 번 backward 불가능
# out2.backward()

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

In [39]:
a = torch.scalar_tensor(2, device=device, requires_grad=True)
b = torch.scalar_tensor(3, device=device, requires_grad=True)
c = torch.scalar_tensor(5, device=device, requires_grad=True)
out1 = a * b
out2 = out1 + c

In [42]:
out2.backward(retain_graph=True)
out2.backward(retain_graph=True)
out2.backward(retain_graph=True) # 여러 번 실행 가능

### Optimizer zero grad ([Document](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html#use-parameter-grad-none-instead-of-model-zero-grad-or-optimizer-zero-grad))

`optimizer.zero_grad()` 보다 `optimizer.zero_grad(set_to_none=True)`가 빠르다고 함. (pytorch >= 1.7)  
0으로 바꾸는 것은 메모리를 비우는 것이 아니라 `None`으로 바꿔 주어야 한다. 